In [25]:
import numpy as np
import pandas as pd 
import math
from scipy.stats import norm

# Black Scholes Pricing Model

In [28]:
def options(K, S, r, dT, sigma, opt_type):
    d1 = (np.log(S/K)+(r+sigma**2/2)*dT)/(sigma*np.sqrt(dT))
    d2 = d1 - sigma*np.sqrt(dT)
    if opt_type == 'Call':
        price = S*norm.cdf(d1)-K*np.exp(-r*dT)*norm.cdf(d2)
    elif opt_type == 'Put':
        price = K*np.exp(-r*dT)*norm.cdf(-d2)-S*norm.cdf(-d1)
    else: 
        print("Invalid type")
    return price 
        

# Determining Greeks

In [33]:
def greeks(K,S,r, T, sigma, opt_type):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T)/(sigma*np.sqrt(T))
    pdf_d1 = norm.pdf(d1)  #N'(d1)
    cdf_d1 = norm.cdf(d1) #N(d1)
    d2 = d1 - sigma*np.sqrt(T) 
    pdf_d2 = norm.pdf(d2)  #N'(d2)
    cdf_d2 = norm.cdf(d2)  #N(d2)

    # computing gamma 
    gamma = pdf_d1/(S*sigma*np.sqrt(T))

    #computing vega 
    vega = S*pdf_d1*np.sqrt(T)

    # if option is a call option 
    if opt_type == 'Call':
        delta = cdf_d1
        theta = -(S*pdf_d1*sigma)/(2*np.sqrt(T))-K*r*np.exp(-r*T)*cdf_d2
        rho = K*T*np.exp(-r*T)*cdf_d2

    # if option is a put option 
    elif opt_type == 'Put':
        delta = norm.cdf(-d1)
        theta = -(S*pdf_d1*sigma)/(2*np.sqrt(T))+K*r*np.exp(-r*T)*norm.cdf(-d2)
        rho = -K*T*np.exp(-r*T)*norm.cdf(-d2)


    return {
        'Delta': delta,
        'Gamma': gamma,
        'Theta (per day)': theta / 365,
        'Vega (per 1% vol)': vega / 100,
        'Rho (per 1% rate)': rho / 100
    }

In [37]:
greeks(20,30,0.01, 2, 0.1, 'Put')

{'Delta': 0.0010377749364545925,
 'Gamma': 0.0008210956741325848,
 'Theta (per day)': -9.23536114145781e-06,
 'Vega (per 1% vol)': 0.001477972213438653,
 'Rho (per 1% rate)': -0.0006480474339290626}